In [177]:
import os
import re
import glob
import pickle
from collections import Counter

import matplotlib.pyplot as plt
from pandas import DataFrame
import pandas as pd
from func import *

data_path = "/home/jichao/MongoDB/reddit/exps/tokenize/ovrw"

In [188]:
def diff_words(sr, size=12):
    fn_wildcard = os.path.join(data_path, sr + "*.pickle")
    filenames = glob.glob(fn_wildcard)
    filenames = sorted(filenames)

    corpus_bg = Counter()
    for fn in filenames:
    #    print fn
        bymonth = pickle.load(open(fn))
        corpus_bg.update(bymonth)

    n_0 = sum(corpus_bg.values())

    result = []

    # print len(filenames) - size + 1
    for i in range(len(filenames) - size + 1):
    #    print i
        corpus_i = Counter()
        for j in range(i, i + size):
            bymonth = pickle.load(open(filenames[j]))
            corpus_i.update(bymonth)        

        n_i = sum(corpus_i.values())

        corpus_j = corpus_bg.copy()
        corpus_j.subtract(corpus_i)
        n_j = sum(corpus_j.values())

        common = set(corpus_i.keys()) & set(corpus_j.keys())
        common = list(common)

        df = [(w, zscore(w, 
                        counter_i=corpus_i,
                        n_i=n_i,
                        counter_j=corpus_j,
                        n_j=n_j,
                        counter_0=corpus_bg,
                        n_0=n_0)) for w in common]

        df = pd.DataFrame(np.array(zip(*df)).T, columns=["word", "zscore"])
        df["zscore"] = df["zscore"].astype(np.float)
        df = df.sort_values(by="zscore")

        under = df.head()
        under = under.reset_index(drop=True)
        over = df.tail()
        over = over.reset_index(drop=True)

        words = pd.concat([under, over], axis=1)

        bn = os.path.basename(filenames[i])
        bn = bn.split(".")[0]
        _, yyyy, mm = bn.split("_")
        start = "-".join([yyyy, mm])
    
        bn = os.path.basename(filenames[i + size -1 ])
        bn = bn.split(".")[0]
        _, yyyy, mm = bn.split("_")
        end = "-".join([yyyy, mm])
    
        result.append((" to ".join([start, end]), words, df["zscore"].describe()))
        
    return result     

In [189]:
men = diff_words("MensRights")

In [192]:
fem = diff_words("Feminism")

In [180]:
def summary_diff_words(result):
    index = pd.Index([result[i][0] for i in range(len(result))])

    overrepr = pd.concat([result[i][1].iloc[:, [2]].T for i in range(len(result))])
    overrepr = overrepr.reset_index(drop=True)
    overrepr.columns = ["overrepr_%d" % i for i in range(1, 6)]
    
    stats = pd.concat([DataFrame(result[i][2]).T for i in range(len(result))])
    stats = stats.reset_index(drop=True)
    
    df = pd.concat([overrepr, stats], axis=1)
    df.index = index
    return df

In [221]:
df_men.iloc[-20:, 0:5]

,overrepr_1,overrepr_2,overrepr_3,overrepr_4,overrepr_5
2014-01 to 2014-12,commenter,rodger,delete,nsfw,toggle
2014-02 to 2015-01,rodger,commenter,delete,nsfw,toggle
2014-03 to 2015-02,feminism,commenter,delete,nsfw,toggle
2014-04 to 2015-03,feminist,rodger,nsfw,gamergate,toggle
2014-05 to 2015-04,elliot,nsfw,rodger,toggle,gamergate
2014-06 to 2015-05,nsfw,sjw,trp,toggle,gamergate
2014-07 to 2015-06,nsfw,shirt,toggle,sjw,gamergate
2014-08 to 2015-07,shirt,pao,toggle,sjw,gamergate
2014-09 to 2015-08,sjws,shirt,pao,sjw,gamergate
2014-10 to 2015-09,sjws,shirt,pao,sjw,gamergate


In [222]:
df_fem.iloc[-20:, 0:5]

,overrepr_1,overrepr_2,overrepr_3,overrepr_4,overrepr_5
2014-01 to 2014-12,toggle,game,player,school,wnba
2014-02 to 2015-01,player,school,toggle,game,wnba
2014-03 to 2015-02,nsfw,school,game,toggle,wnba
2014-04 to 2015-03,video,toggle,school,game,wnba
2014-05 to 2015-04,toggle,player,video,game,wnba
2014-06 to 2015-05,zoe,video,player,game,wnba
2014-07 to 2015-06,player,game,zoe,dawood,wnba
2014-08 to 2015-07,sen,gamergate,zoe,dawood,pao
2014-09 to 2015-08,quran,isi,gamergate,pao,dawood
2014-10 to 2015-09,isi,quran,gamergate,pao,dawood


In [199]:
df_men = summary_diff_words(men)
df_fem = summary_diff_words(fem)
a = df_men.ix["2012-06 to 2013-05":"2013-06 to 2014-05", 0:5]
b = df_fem.ix["2012-06 to 2013-05":"2013-06 to 2014-05", 0:5]

In [217]:
pd.concat([a, b], axis=1, keys=["MensRights", "Feminism"])

MensRights                                      \
                    overrepr_1  overrepr_2  overrepr_3  overrepr_4   
2012-06 to 2013-05       adria         srs        text        copy   
2012-07 to 2013-06       adria         srs        text        copy   
2012-08 to 2013-07         gww      poster        text        copy   
2012-09 to 2013-08        rape  patriarchy        text        copy   
2012-10 to 2013-09       adria  patriarchy        text          sr   
2012-11 to 2013-10    feminist        rape        text  patriarchy   
2012-12 to 2013-11        text        rape        copy    feminist   
2013-01 to 2013-12       adria    feminist        text        copy   
2013-02 to 2014-01    feminist        text        copy       adria   
2013-03 to 2014-02        text        copy        geek  patriarchy   
2013-04 to 2014-03    textwall        text  patriarchy        copy   
2013-05 to 2014-04  patriarchy   zimmerman        text        copy   
2013-06 to 2014-05    feminist         mrm        text        copy   

                                 Feminism                                \
                    overrepr_5 overrepr_1     overrepr_2     overrepr_3   
2012-06 to 2013-05          sr     rapist            men             sr   
2012-07 to 2013-06          sr        men         rapist           tosh   
2012-08 to 2013-07          sr       nerd         rapist           geek   
2012-09 to 2013-08          sr      femen        picture           geek   
2012-10 to 2013-09        copy        soy          femen        consent   
2012-11 to 2013-10        copy      false         rapist        consent   
2012-12 to 2013-11  patriarchy       song        consent          cyrus   
2013-01 to 2013-12  patriarchy     thicke           rape  objectionable   
2013-02 to 2014-01  patriarchy      femen         thicke  objectionable   
2013-03 to 2014-02       adria     thicke          allen  objectionable   
2013-04 to 2014-03        geek       song         thicke  objectionable   
2013-05 to 2014-04        geek       song  objectionable         thicke   
2013-06 to 2014-05        geek      allen           song         thicke   

                                          
                   overrepr_4 overrepr_5  
2012-06 to 2013-05       joke       rape  
2012-07 to 2013-06       joke       rape  
2012-08 to 2013-07    picture       rape  
2012-09 to 2013-08     rapist       rape  
2012-10 to 2013-09      false       rape  
2012-11 to 2013-10      miley       rape  
2012-12 to 2013-11       rape      miley  
2013-01 to 2013-12      cyrus      miley  
2013-02 to 2014-01      cyrus      miley  
2013-03 to 2014-02      cyrus      miley  
2013-04 to 2014-03      cyrus      miley  
2013-05 to 2014-04      cyrus      miley  
2013-06 to 2014-05      cyrus      miley